## Diabetes

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import numpy as np

device = torch.device('cpu')

In [2]:
from sklearn.datasets import load_diabetes

data = load_diabetes()
print(data.feature_names)

X, y = data.data, data.target

['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']


## 1. Specifying input and targets

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [4]:
train_input  = torch.tensor(X_train, dtype=torch.float32)
test_input   = torch.tensor(X_test,  dtype=torch.float32)
train_target = torch.tensor(y_train, dtype=torch.float32)
test_target  = torch.tensor(y_test,  dtype=torch.float32)

train_input.shape, test_input.shape, train_target.shape, test_target.shape

(torch.Size([296, 10]),
 torch.Size([146, 10]),
 torch.Size([296]),
 torch.Size([146]))

## 2. Dataloaders

In [5]:
# print(data.feature_names)
train_ds = TensorDataset(train_input, train_target)
test_ds  = TensorDataset(test_input, test_target)
# train_ds[0]

In [6]:
batch_size = 32 #binary - fits nicely to CPU / GPU -- #296 / 32 = 10 rounds
train_dl   = DataLoader(train_ds, batch_size = batch_size, shuffle=True, num_workers=4)
test_dl    = DataLoader(test_ds,  batch_size = batch_size, shuffle=True, num_workers=4)

In [7]:
# #test
# for batch_x, batch_y in train_dl:
#     print("One batch of X: ", batch_x.shape)
#     print("One batch of y: ", batch_y.shape)
#     break

## 3. Layers

In [8]:
model = nn.Sequential(
    nn.Linear(10, 24),
    nn.ReLU(),  #gradient = 1
    nn.Linear(24, 12),
    nn.ReLU(), 
    nn.Linear(12, 6),
    nn.ReLU(),
    nn.Linear(6, 1)
)

In [9]:
model[0].weight.dtype

torch.float32

In [10]:
total_num_of_params = 0
for ix, p in enumerate(model.parameters()):
    total_num_of_params += p.numel()
total_num_of_params

649

In [11]:
train_input.shape

torch.Size([296, 10])

In [12]:
#always good to test your neural network before training
yhat = model(train_input)
assert yhat.shape[1] == 1

## 4. Loss function

In [13]:
criterion = nn.MSELoss()

## 5. Optimizer

In [14]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

## 5. Training

In [15]:
num_epochs = 5

for epoch in range(num_epochs):
    for batch_x, batch_y in train_dl:
        batch_x.to(device)
        batch_y.to(device)
        batch_y = batch_y.reshape(-1, 1) #(m, ) ==> (m, 1)
        
        yhat = model(batch_x)
        
        loss = criterion(yhat, batch_y)
        
        optimizer.zero_grad() 
        loss.backward()
        
        optimizer.step()

    print(f"Epoch: {epoch} | Loss: {loss:.2f}")

Epoch: 0 | Loss: 33092.90
Epoch: 1 | Loss: 19094.50
Epoch: 2 | Loss: 2724.31
Epoch: 3 | Loss: 8133.44
Epoch: 4 | Loss: 6114.92


In [16]:
# #save your model
filename = 'model/diabetes.pth'
torch.save(model.state_dict(), filename)

## 6. Testing

In [21]:
model.eval()  #change the model to eval mode - it will skip dropout, batch norm
total_avg_mse = 0
with torch.no_grad():
    
    for batch_x, batch_y in test_dl:
        
        yhat = model(batch_x)
        batch_y = batch_y.reshape((-1, 1))
        mse  = criterion(yhat, batch_y)
        
        total_avg_mse += mse.item() / len(test_dl)
        
print("Total Average MSE: ", total_avg_mse)
        

Total Average MSE:  5934.153124999999
